In [1]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import easyquotation
import math
import numpy as np
import pandas as pd
from easyquant import MongoIo
# from easyquant.qafetch import QATdx as tdx
from easyquant.indicator.base import *
import pandas as pd
from easyquant import EasyTime
from datetime import datetime, time
from ctypes import * 
# import datetime
import math
import numpy as np
import pandas as pd
import talib
from ctypes import * 
import QUANTAXIS as QA

# import time

In [32]:
m=MongoIo()
source=easyquotation.use('sina')

In [131]:
table = 'st_orders-%s' % 'tdx_yhzc'
code='002614'

In [133]:
newdata=source.stocks(code)[code]

In [134]:
now_price = newdata['now']

In [110]:

def new_df(df_day, data, now_price):
    code = data['code']
    # now_vol = data['volume']
    last_time = pd.to_datetime(data['datetime'][0:10])
    # print("code=%s, data=%s" % (self.code, self._data['datetime']))
    # df_day = data_buf_day[code]
    # df_day.loc[last_time]=[0 for x in range(len(df_day.columns))]
    df_day.at[(last_time, code), 'open'] = data['open']
    df_day.at[(last_time, code), 'high'] = data['high']
    df_day.at[(last_time, code), 'low'] = data['low']
    df_day.at[(last_time, code), 'close'] = now_price
    df_day.at[(last_time, code), 'volume'] = data['volume']
    df_day.at[(last_time, code), 'amount'] = data['amount']
    return df_day


In [142]:
def tdx_dqe_cfc_A1(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC =IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (C / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
    if sort:
        刀 = (MC - JC) / JC * 1000
    else:
        刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [143]:
def tdx_dqe_cfc_A12(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC =IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (O / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
    if sort:
        刀 = (MC - JC) / JC * 1000
    else:
        刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [135]:
data = new_df(data.copy(), newdata, now_price)

In [151]:
# today
code='300063'
data=m.get_stock_day(code)
newdata=source.stocks(code)[code]
now_price = newdata['now']
data = new_df(data.copy(), newdata, now_price)
data.iat[-1,3]=newdata['open']
data.tail()
C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low


,,open,high,low,close,volume,amount
date,code,,,,,,
2020-11-02,300063,7.55,7.76,7.21,7.35,4.516050e+05,335985120.0
2020-11-03,300063,7.50,7.60,7.30,7.53,4.213430e+05,315340608.0
2020-11-04,300063,7.50,7.53,7.11,7.21,4.484330e+05,326091040.0
2020-11-05,300063,7.35,7.61,7.30,7.43,4.439240e+05,330125216.0
2020-11-06,300063,7.90,8.19,7.48,7.90,5.602389e+08,72013755.0


In [155]:
TJ1=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False).tail()
TJ2=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0).tail()
tdx_dqe_cfc_A1(data).tail()

date        code  
2020-11-02  300063    -0.000000
2020-11-03  300063    -6.815362
2020-11-04  300063    -0.000000
2020-11-05  300063     1.122186
2020-11-06  300063    30.333110
dtype: float64

In [156]:
TJ1

date        code  
2020-11-02  300063    False
2020-11-03  300063    False
2020-11-04  300063    False
2020-11-05  300063    False
2020-11-06  300063     True
dtype: bool

In [157]:
TJ2

date        code  
2020-11-02  300063    True
2020-11-03  300063    True
2020-11-04  300063    True
2020-11-05  300063    True
2020-11-06  300063    True
dtype: bool

In [ ]:
300208    14.250414
300538    33.348295
300050    20.557449
002614    24.505901
300063    30.333110

In [170]:
# back:
code='300464'
data=m.get_stock_day(code)

C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low
data.tail()

,,open,high,low,close,volume,amount
date,code,,,,,,
2020-10-30,300464,19.71,19.82,18.74,18.90,85436.0,164673808.0
2020-11-02,300464,18.85,19.19,18.19,18.73,58826.0,109639112.0
2020-11-03,300464,18.79,19.50,18.72,19.26,64459.0,123776456.0
2020-11-04,300464,19.36,19.44,18.80,19.19,60066.0,114918648.0
2020-11-05,300464,20.00,21.80,19.87,20.76,236941.0,488547616.0


In [171]:
TJ1=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False).tail()
TJ2=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0).tail()
tdx_dqe_cfc_A12(data).tail()

date        code  
2020-10-30  300464    -0.000000
2020-11-02  300464    -0.000000
2020-11-03  300464     2.975439
2020-11-04  300464     3.961511
2020-11-05  300464    20.358610
dtype: float64

In [172]:
TJ1

date        code  
2020-10-30  300464    False
2020-11-02  300464    False
2020-11-03  300464    False
2020-11-04  300464    False
2020-11-05  300464     True
dtype: bool

In [173]:
TJ2

date        code  
2020-10-30  300464    True
2020-11-02  300464    True
2020-11-03  300464    True
2020-11-04  300464    True
2020-11-05  300464    True
dtype: bool